Query 1: Give the name of managers living currently in Houston

In [5]:
select e.fname as name, e.lname as surname
from employee e
join department d on e.SSN = d.MgrSSN
join employeeAddress ed on e.SSN = ed.SSN
where ed.city = 'Houston' 
    AND cast(ed.fromDate as date) <= cast(getdate() as date)
    AND cast(ed.toDate as date) > cast(getDate() as date)
    AND cast(d.fromDate as date) <= cast(getdate() as date)
    AND cast(d.toDate as date) > cast(getDate() as date)

(2 rows affected)

Total execution time: 00:00:00.060

name,surname
James,Borg
Franklin,Wong


Query 2: Give the name of employees working currently in the ‘Research’ department and having a salary greater or equal than 45000

In [6]:
select e.fname as name, e.lname as surname
from employee e
join affiliation a on a.SSN = e.SSN 
join department d on a.DNumber = d.DNumber
join employeeSalary es on es.SSN = e.SSN
where d.DName = 'Research'
    AND es.salary >= 45000
    AND cast(es.fromDate as date) <= cast(getdate() as date)
    AND cast(es.toDate as date) > cast(getDate() as date)
    AND cast(a.fromDate as date) <= cast(getdate() as date)
    AND cast(a.toDate as date) > cast(getDate() as date);

(1 row affected)

Total execution time: 00:00:00.050

name,surname
Franklin,Wong


Query 3: Give the name of current employees who do not work currently in any department

In [7]:
select e.fname as name, e.lname as surname
from employee e
join employeeLifecycle el on el.SSN = e.SSN 
where cast(el.fromDate as date) <= cast(getdate() as date)
    AND cast(el.toDate as date) > cast(getDate() as date)
    AND 
    not exists (
    select *
    from affiliation a
    where a.SSN = e.SSN 
    AND cast(a.fromDate as date) <= cast(getdate() as date)
    AND cast(a.toDate as date) > cast(getDate() as date)
);

(0 rows affected)

Total execution time: 00:00:00.032

name,surname


Query 4: Give the name of the employee(s) that had the highest salary on 1/1/2002

In [8]:
select e.fname as name, e.lname as surname, es.salary as salary
from employee e 
join employeeSalary es on es.SSN = e.SSN 
where es.salary = (
    select max(salary)
    from employeeSalary es2
    where cast(es2.fromDate as date) <= cast('01-01-2002' as date)
    AND cast(es2.toDate as date) > cast('01-01-2002' as date)
)
    AND cast(es.fromDate as date) <= cast('01-01-2002' as date)
    AND cast(es.toDate as date) > cast('01-01-2002' as date);

(1 row affected)

Total execution time: 00:00:00.015

name,surname,salary
James,Borg,58000


Query 5: Provide the salary and affiliation history for all employees

In [9]:
-- min of two dates
create function minDate
(@one smalldatetime, @two smalldatetime)
returns smalldatetime as
begin
return CASE WHEN @one < @two then @one else @two end
end
go
-- max of two dates
create function maxDate
(@one smalldatetime, @two smalldatetime)
returns smalldatetime as
begin
return CASE WHEN @one > @two then @one else @two end
end
go

Commands completed successfully.

Commands completed successfully.

(12 rows affected)

Total execution time: 00:00:00.024

name,surname,dpt,salary
Ahmad,Jabbar,Administration,25000
Alicia,Zelaya,Administration,25000
Franklin,Wong,Administration,40000
Franklin,Wong,Administration,45000
Franklin,Wong,Research,45000
James,Borg,Headquarters,55000
James,Borg,Headquarters,58000
Jennifer,Wallace,Administration,43000
John,Smith,Headquarters,30000
John,Smith,Research,30000


In [25]:
-- query
select e.fname as name, e.lname as surname, d.dname as dpt, es.salary, dbo.maxDate(a.fromDate,es.fromDate) as fromDate, dbo.minDate(a.toDate, es.toDate) as toDate
from employee e 
join affiliation a on a.SSN = e.SSN
join department d on d.DNumber = a.DNumber
join employeeSalary es on es.SSN = e.SSN
where dbo.maxDate(a.fromDate,es.fromDate) < dbo.minDate(a.toDate, es.toDate)
order by name, surname

(12 rows affected)

Total execution time: 00:00:00.011

name,surname,dpt,salary,fromDate,toDate
Ahmad,Jabbar,Administration,25000,1985-01-01 00:00:00,2079-01-01 00:00:00
Alicia,Zelaya,Administration,25000,1985-01-01 00:00:00,2079-01-01 00:00:00
Franklin,Wong,Administration,40000,1982-01-01 00:00:00,1983-01-01 00:00:00
Franklin,Wong,Administration,45000,1983-01-01 00:00:00,1984-01-01 00:00:00
Franklin,Wong,Research,45000,1984-01-01 00:00:00,2079-01-01 00:00:00
James,Borg,Headquarters,55000,1980-01-01 00:00:00,1981-01-01 00:00:00
James,Borg,Headquarters,58000,1981-01-01 00:00:00,2079-01-01 00:00:00
Jennifer,Wallace,Administration,43000,1982-01-01 00:00:00,2079-01-01 00:00:00
John,Smith,Headquarters,30000,1985-01-01 00:00:00,1986-01-01 00:00:00
John,Smith,Research,30000,1986-01-01 00:00:00,2079-01-01 00:00:00


Query 6. Give the name of employees and the period of time in which they were supervisors but did not work in any project during the same period.

In [1]:
--Case 1: periods in which the employee is supervisor, and was not working in any project at the beginning,
-- but started working in some project at some point
select S.supervisor, S.FromDate, W1.FromDate as ToDate
from Supervision S, WorksOn W1
where S.supervisor = W1.SSN
and S.FromDate < W1.FromDate and W1.FromDate < S.ToDate
and not exists ( select * from WorksOn W2 where S.supervisor = W2.SSN
and S.FromDate < W2.ToDate and W2.FromDate < W1.FromDate )
union
--Case 2: periods in which the employee is supervisor, and worked in some project at the beginning, but
-- stopped working in any project at some point
select S.supervisor, W1.ToDate as FromDate, S.ToDate
from Supervision S, WorksOn W1
where S.supervisor = W1.SSN
and S.FromDate < W1.ToDate and W1.ToDate < S.ToDate
and not exists ( select * from WorksOn W2 where S.supervisor = W2.SSN
and W1.ToDate < W2.ToDate and W2.FromDate < S.ToDate )
union
--Case 3: periods in which the employee is supervisor, and worked in some project at the beginning and in the end,
-- but did not work in any project in between
select S.supervisor, W1.ToDate as FromDate, W2.FromDate as ToDate
from Supervision S, WorksOn W1, WorksOn W2
where S.supervisor = W1.SSN and S.supervisor = W2.SSN and W1.ToDate < W2.FromDate
and S.FromDate < W1.ToDate and W2.FromDate < S.ToDate
and not exists ( select * from WorksOn W3 where S.supervisor = W3.SSN
and W1.ToDate < W3.ToDate and W3.FromDate < W2.FromDate )
union
--Case 4: periods in which the employee is supervisor, and has not worked in a project in the whole period
select supervisor, FromDate, ToDate from Supervision S
where not exists ( select * from WorksOn W where S.supervisor=W.SSN
and S.FromDate < W.ToDate and W.FromDate < S.ToDate )

(3 rows affected)

Total execution time: 00:00:00.045

supervisor,FromDate,ToDate
333445555,2002-01-01,2005-01-01
888665555,1982-01-01,1983-01-01
987654321,2002-01-01,2079-01-01


Query 7. Give the name of supervisors who had work on a project at some time.

In [6]:
select distinct e.fname as name, e.lname as surname
from employee e 
join supervision s on s.supervisor = e.SSN
join worksOn w on w.SSN = e.SSN
order by name, surname

(3 rows affected)

Total execution time: 00:00:00.004

name,surname
Franklin,Wong
James,Borg
Jennifer,Wallace


Query 8.  Give the name of employees and the date they changed their affiliation.

In [7]:
select e.fname as name, e.lname as surname, a1.toDate as changeDate
from employee e
join affiliation a1 on a1.SSN = e.SSN
join affiliation a2 on a2.SSN = e.SSN
where a1.toDate = a2.fromDate and a1.DNumber <> a2.DNumber

(2 rows affected)

Total execution time: 00:00:00.013

name,surname,changeDate
John,Smith,1986-01-01
Franklin,Wong,1984-01-01


Query 9.  Give the name of employees and the periods they worked on any project

In [22]:
select distinct e.Fname as name, e.LName as surname, w1.fromDate as fromDate, w2.toDate as toDate
from employee e
join worksOn w1 on w1.SSN = e.SSN
join worksOn w2 on w2.SSN = e.SSN
where w1.fromDate < w2.toDate and w2.fromDate <= w1.toDate
    and 
    not exists(
        select * from worksOn w3
        where w3.SSN = w1.SSN and w3.fromDate < w1.fromDate and w3.toDate >= w1.fromDate
    )
    and
    not exists(
        select * from worksOn w3
        where w3.SSN = e.SSN and w3.fromDate <= w2.toDate and w3.toDate > w2.toDate
    )

(9 rows affected)

Total execution time: 00:00:00.013

name,surname,fromDate,toDate
Ahmad,Jabbar,1985-01-01,2079-01-01
Alicia,Zelaya,1985-01-01,2079-01-01
Franklin,Wong,1982-01-01,2002-01-01
Franklin,Wong,2005-01-01,2079-01-01
James,Borg,1983-01-01,2079-01-01
Jennifer,Wallace,1982-01-01,2002-01-01
John,Smith,1985-01-01,2079-01-01
Joyce,English,1985-01-01,2079-01-01
Ramesh,Narayan,1985-01-01,2079-01-01


In [19]:
-- professor's solution
select distinct E.SSN, E.FName, E.LName, F.FromDate, L.ToDate
from Employee E, WorksOn F, WorksOn L
where E.SSN = F.SSN and F.SSN = L.SSN and F.FromDate < L.ToDate
and not exists ( select * from WorksOn M
                where M.SSN = F.SSN
                and F.FromDate < M.FromDate and M.FromDate <= L.ToDate
                and not exists ( select * from WorksOn T1
                where T1.SSN = F.SSN
                and T1.FromDate < M.FromDate and M.FromDate <= T1.ToDate ) )
and not exists ( select * from WorksOn T2
                where T2.SSN = F.SSN
                and ( ( T2.FromDate < F.FromDate and F.FromDate <= T2.ToDate )
                or ( T2.FromDate <= L.ToDate and L.ToDate < T2.ToDate ) ) )
order by e.Fname, e.LName

(9 rows affected)

Total execution time: 00:00:00.007

SSN,FName,LName,FromDate,ToDate
987987987,Ahmad,Jabbar,1985-01-01,2079-01-01
999887777,Alicia,Zelaya,1985-01-01,2079-01-01
333445555,Franklin,Wong,1982-01-01,2002-01-01
333445555,Franklin,Wong,2005-01-01,2079-01-01
888665555,James,Borg,1983-01-01,2079-01-01
987654321,Jennifer,Wallace,1982-01-01,2002-01-01
123456789,John,Smith,1985-01-01,2079-01-01
453453453,Joyce,English,1985-01-01,2079-01-01
666884444,Ramesh,Narayan,1985-01-01,2079-01-01


Query 10. Give the history of the maximum salary

In [21]:
with Instants(Instant) AS ( -- All time points in salary timeline
            select distinct E.FromDate from EmployeeSalary E
            union
            select distinct E.ToDate from EmployeeSalary E),
    Intervals(FromDate,ToDate) AS ( -- All consecutive instants
            select distinct I1.Instant, I2.Instant
            from Instants I1, Instants I2
            where I1.Instant < I2.Instant
            and not exists ( select *
                            from Instants I3
                            where I1.Instant < I3.Instant
                            and I3.Instant < I2.Instant ) ),
    TempMax(SalaryMax, FromDate, ToDate) AS ( -- Obtain the max salary in each interval by computing the maximum
                                            -- of all salaries that are active in each interval
            select max(E.Salary), I.FromDate, I.ToDate
            from EmployeeSalary E, Intervals I
            where E.FromDate <= I.FromDate -- E was earning the salary before I started
            and I.ToDate <= E.ToDate -- E continued ending the salary after I started
            group by I.FromDate, I.ToDate )
select distinct F.SalaryMax, F.FromDate, L.ToDate -- The actual select (Coalesce values)
    from TempMax F join TempMax L on L.salaryMax = F.salaryMax -- Periods of same max salary
    where F.FromDate < L.ToDate -- F starts before L finishes
    and not exists ( select * -- Every interval M starting after L started and before F ended
                              -- must start inside a single interval T1
                    from TempMax M
                    where M.SalaryMax = F.SalaryMax
                        and F.ToDate < M.FromDate and M.FromDate <= L.FromDate
                        and not exists ( select *
                                        from TempMax T1
                                        where T1.SalaryMax = F.SalaryMax
                                        and T1.FromDate < M.FromDate and M.FromDate <= T1.ToDate ) )
    and not exists ( select * -- There is no T2 extending F to the left nor L to the right
                    from TempMax T2
                    where T2.SalaryMax = F.SalaryMax
                    and ( ( T2.FromDate < F.FromDate and F.FromDate <= T2.ToDate )
                    or ( T2.FromDate <= L.ToDate and L.ToDate < T2.ToDate ) ) )
    order by F.FromDate

(2 rows affected)

Total execution time: 00:00:00.038

SalaryMax,FromDate,ToDate
55000,1980-01-01,1981-01-01
58000,1981-01-01,2079-01-01


Query 11. Give by department the history of the maximum salary

In [27]:
with Instants(Instant) AS (
            select distinct E.FromDate from EmployeeSalary E
            union
            select distinct E.ToDate from EmployeeSalary E),
    Intervals(FromDate,ToDate) AS (
            select distinct I1.Instant, I2.Instant
            from Instants I1, Instants I2
            where I1.Instant < I2.Instant
            and not exists ( select *
                            from Instants I3
                            where I1.Instant < I3.Instant
                            and I3.Instant < I2.Instant ) ),
    TempMax(DNumber, SalaryMax, FromDate, ToDate) AS (
            select a.DNumber, max(E.Salary), I.FromDate, I.ToDate
            from affiliation a, EmployeeSalary E, Intervals I -- Add the affiliation
            where a.SSN = E.SSN and E.FromDate <= I.FromDate -- join by SSN
            and I.ToDate <= E.ToDate
            group by a.DNumber, I.FromDate, I.ToDate ) -- group by DNumber
select distinct F.DNumber, F.SalaryMax, F.FromDate, L.ToDate 
    from TempMax F join TempMax L on L.salaryMax = F.salaryMax 
    where F.FromDate < L.ToDate and F.DNumber = L.DNumber -- Join also by Department
    and not exists ( select * 
                    from TempMax M
                    where M.SalaryMax = F.SalaryMax and M.DNumber = F.DNumber -- Join also by department
                        and F.ToDate < M.FromDate and M.FromDate <= L.FromDate
                        and not exists ( select *
                                        from TempMax T1
                                        where T1.SalaryMax = F.SalaryMax and T1.DNumber = F.DNumber -- Join also by dpt
                                        and T1.FromDate < M.FromDate and M.FromDate <= T1.ToDate ) )
    and not exists ( select *
                    from TempMax T2
                    where T2.SalaryMax = F.SalaryMax and T2.DNumber = F.DNumber -- Join also by dpt
                    and ( ( T2.FromDate < F.FromDate and F.FromDate <= T2.ToDate )
                    or ( T2.FromDate <= L.ToDate and L.ToDate < T2.ToDate ) ) )
    order by DNumber, F.FromDate

(6 rows affected)

Total execution time: 00:00:00.042

DNumber,SalaryMax,FromDate,ToDate
1,55000,1980-01-01,1981-01-01
1,58000,1981-01-01,2079-01-01
4,43000,1982-01-01,1983-01-01
4,45000,1983-01-01,2079-01-01
5,40000,1982-01-01,1983-01-01
5,45000,1983-01-01,2079-01-01


In [28]:
WITH Aff_Sal (DNumber, Salary, FromDate, ToDate) AS ( -- Temporal join of affiliation and salary
        select distinct A.DNumber, S.Salary, dbo.maxDate(S.FromDate,A.FromDate), dbo.minDate(S.ToDate,A.ToDate)
        from Affiliation A, EmployeeSalary S
        where A.SSN = S.SSN
        and dbo.maxDate(S.FromDate,A.FromDate) < dbo.minDate(S.ToDate,A.ToDate) ),
    SalChanges(DNumber, Instant) AS (
        select distinct DNumber, FromDate from Aff_Sal
        union select distinct DNumber, ToDate from Aff_Sal ),
SalIntervals(DNumber, FromDate, ToDate) AS (
select distinct P1.DNumber, P1.Instant, P2.Instant
from SalChanges P1, SalChanges P2
where P1.DNumber=P2.DNumber and P1.Instant<P2.Instant
and not exists ( select * from SalChanges P3
where P1.DNumber = P3.DNumber and P1.Instant < P3.Instant
and P3.Instant < P2.Instant ) ),
-- Second step: Compute the maximum salary for the
-- above periods.
TempMaxDep(DNumber, MaxSalary, FromDate, ToDate) AS (
select P.DNumber, max(Salary), P.FromDate, P.ToDate
from Aff_Sal A, SalIntervals P
where A.DNumber = P.DNumber
and A.FromDate <= P.FromDate and P.ToDate <= A.ToDate
group by P.DNumber, P.FromDate, P.ToDate )
-- Third step: Coalescing the above table
select distinct F.DNumber, F.MaxSalary, F.FromDate, L.ToDate
from TempMaxDep F, TempMaxDep L
where F.DNumber = L.DNumber and F.MaxSalary = L.MaxSalary
and F.FromDate < L.ToDate
and not exists ( select *
from TempMaxDep M
where F.DNumber = M.DNumber and F.MaxSalary = M.MaxSalary
and F.ToDate < M.FromDate and M.FromDate <= L.FromDate
and not exists ( select *
from TempMaxDep T1
where F.DNumber = T1.DNumber and F.MaxSalary = T1.MaxSalary
and T1.FromDate < M.FromDate and M.FromDate <= T1.ToDate ) )
and not exists ( select *
from TempMaxDep T2
where F.DNumber = T2.DNumber and F.MaxSalary = T2.MaxSalary
and ( ( T2.FromDate < F.FromDate and F.FromDate <= T2.ToDate )
or ( T2.FromDate <= L.ToDate and L.ToDate < T2.ToDate ) ) )
order by F.DNumber, F.FromDate

(6 rows affected)

Total execution time: 00:00:00.181

DNumber,MaxSalary,FromDate,ToDate
1,55000,1980-01-01 00:00:00,1981-01-01 00:00:00
1,58000,1981-01-01 00:00:00,2079-01-01 00:00:00
4,43000,1982-01-01 00:00:00,1983-01-01 00:00:00
4,45000,1983-01-01 00:00:00,1984-01-01 00:00:00
4,43000,1984-01-01 00:00:00,2079-01-01 00:00:00
5,45000,1984-01-01 00:00:00,2079-01-01 00:00:00
